## Data Processing

In [ ]:
import pandas as pd

In [ ]:
bucket = ""

In [ ]:
#file = "cleaned_historical_trips_2015_2022.csv"

#s3_data_location = f"s3://{bucket}/{file}*"
#trips = pd.read_csv(s3_data_location, parse_dates = True)

In [ ]:
#trips = trips[(trips["starttime"] > "2017-09-01") & (trips["stoptime"] < "2022-08-31")]

In [ ]:
#trips_start = trips[["starttime", "start station id", "start station name"]]
#trips_stop = trips[["stoptime", "end station id", "end station name"]]

In [ ]:
#trips_start.to_csv("model_trips_start_station_2017_2022.csv")
#trips_stop.to_csv("model_trips_stop_station_2017_2022.csv")

### Trip Start Station

In [ ]:
start_file = "model_trips_start_station_2017_2022.csv"

s3_start_location = f"s3://{bucket}/{start_file}*"
trips_start = pd.read_csv(s3_start_location, parse_dates = True)

In [ ]:
trips_start["start station id"].nunique()

In [ ]:
trips_start["start station name"].nunique()

In [ ]:
trips_start.drop_duplicates(subset = ["start station id", "start station name"]).to_csv("unique_start.csv")

### Trip End Station

In [ ]:
end_file = "model_trips_stop_station_2017_2022.csv"

s3_end_location = f"s3://{bucket}/{end_file}*"
trips_stop = pd.read_csv(s3_end_location, parse_dates = True)